In [22]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import nasnet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [23]:
model_name='NASNetLarge'
number_of_epochs = 50
frozen = False

base_model=nasnet.NASNetLarge(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

if frozen:
    for layer in base_model.layers:
        layer.trainable = False

    
base_model.summary()    
    
x = base_model.output

#These layers act as output layer for mobilenet but are customed to fit for 6 classes instead of imagenets original 1000.
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(6,activation='softmax')(x) #final layer with softmax activation


#%%

model=Model(inputs=base_model.input,outputs=preds)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 331, 331, 3)  0                                            
__________________________________________________________________________________________________
stem_conv1 (Conv2D)             (None, 165, 165, 96) 2592        input_6[0][0]                    
__________________________________________________________________________________________________
stem_bn1 (BatchNormalization)   (None, 165, 165, 96) 384         stem_conv1[0][0]                 
__________________________________________________________________________________________________
activation_1301 (Activation)    (None, 165, 165, 96) 0           stem_bn1[0][0]                   
__________________________________________________________________________________________________
reduction_

                                                                 normal_bn_1_7[0][0]              
__________________________________________________________________________________________________
normal_concat_7 (Concatenate)   (None, 21, 21, 2016) 0           adjust_bn_7[0][0]                
                                                                 normal_add_1_7[0][0]             
                                                                 normal_add_2_7[0][0]             
                                                                 normal_add_3_7[0][0]             
                                                                 normal_add_4_7[0][0]             
                                                                 normal_add_5_7[0][0]             
__________________________________________________________________________________________________
activation_1417 (Activation)    (None, 21, 21, 1344) 0           reduction_concat_reduce_6[0][0]  
__________

separable_conv_1_bn_normal_righ (None, 11, 11, 672)  2688        separable_conv_1_normal_right2_18
__________________________________________________________________________________________________
separable_conv_1_bn_normal_left (None, 11, 11, 672)  2688        separable_conv_1_normal_left5_18[
__________________________________________________________________________________________________
activation_1551 (Activation)    (None, 11, 11, 672)  0           separable_conv_1_bn_normal_left1_
__________________________________________________________________________________________________
activation_1553 (Activation)    (None, 11, 11, 672)  0           separable_conv_1_bn_normal_right1
__________________________________________________________________________________________________
activation_1555 (Activation)    (None, 11, 11, 672)  0           separable_conv_1_bn_normal_left2_
__________________________________________________________________________________________________
activation

In [24]:
datagen = dict(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
		validation_split=0.13,
        fill_mode='nearest')

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, **datagen)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = train_datagen.flow_from_directory('./dataset/train',
                                                 target_size=(331,331),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 subset='training')

validation_generator = train_datagen.flow_from_directory('./dataset/train',
                                                 target_size=(331,331),
                                                 color_mode='rgb',
                                                 batch_size=train_generator.batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 subset='validation')

test_generator = test_datagen.flow_from_directory('./dataset/test',
                                                 target_size=(331,331),
                                                 color_mode='rgb',
                                                 batch_size=1,
                                                 class_mode='categorical',
                                                 shuffle=False
                                                 )

Found 1825 images belonging to 6 classes.
Found 271 images belonging to 6 classes.
Found 431 images belonging to 6 classes.


In [ ]:



#specify the inputs
#specify the outputs
#now a model has been created based on our architecture


#%%

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train = train_generator.n // train_generator.batch_size
step_size_validation = validation_generator.n // train_generator.batch_size


history = model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_data = validation_generator, 
                   validation_steps = step_size_validation,
                   epochs = number_of_epochs)

test_loss, test_acc = model.evaluate_generator(generator=test_generator, steps=test_generator.n)

#Get history of loss and accuracy during training and display it with graphs
train_loss = history.history['loss']
train_acc  = history.history['acc']
val_loss = history.history['val_loss']
val_acc = history.history['val_acc']

Epoch 1/50


In [ ]:
"""
print('train_loss:', train_loss)
print('train_acc:', train_acc)
print('val_loss:', val_loss)
print('val_acc:', val_acc)
"""
print('test_loss:', test_loss)
print('test_acc:', test_acc) 

f= open("test_results.txt","a+")
f.write("____________________________ %s _____________________________ \r\n" % model_name )
f.write("Number of epochs:%d\r\n" % number_of_epochs)
f.write("Frozen: %s\r\n" % str(frozen))
f.write("Test Accuracy:%f\r\n" % test_acc)
f.write("Test Loss:%f\r\n" % test_loss)
f.close()

print('A graph displaying the loss over training epochs')
plt.plot(train_loss)
plt.plot(val_loss)
#plt.axhline(test_loss, color='g')
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch number')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.savefig('graphs/train_loss_' + model_name + '_' + str(number_of_epochs) + 'epochs_' + 'Frozen_' + str(frozen) + '.png')

print('A graph displaying the accuracy over training epochs')
plt.plot(train_acc)
plt.plot(val_acc)
#plt.axhline(test_acc, color='g')
plt.title('Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch number')
plt.legend(['train', 'val', 'test'], loc='upper left')
plt.savefig('graphs/train_acc_' + model_name + '_' + str(number_of_epochs) + 'epochs_' + 'Frozen_' + str(frozen) + '.png')
plt.show()